In [1]:
import re
import numpy as np

In [2]:
test_input='December12_test_input.txt'
with open(test_input) as f :
    file=f.readlines()
    doc=[[x.split()[0], [int(s) for s in re.findall(r'\d+',x)]] for x in file]

In [3]:
test1='????.#...#...'
test2='????.#...#...'

In [4]:

#### Remove . (dots) on left and right of string

def simplify_dots(line,indication):
    line=line.strip('.')
    return line, indication


#### Function to verify if first/last indication is fulfilled
##### we use it on a simplified version of the line (i.e. stripped from the dots).

def simplify_left_correct(line,indication):
    if indication== []:
        return '.'*len(line),indication
    
    line,indication=simplify_dots(line,indication)
    
    if line[:indication[0]]=='#'*indication[0]:
        return line[indication[0]+1:], indication[1:]
    else:
        return line,indication

def simplify_right_correct(line,indication):
    if indication== []:
        return '.'*len(line),indication
    
    line,indication=simplify_dots(line,indication)
    
    if line[-indication[-1]:]=='#'*indication[-1]:
        
        return line[:-indication[-1]], indication[:-1]
    
    else:
        return line, indication


def simplify_both_correct(line,indication):
    if indication== []:
        return '.'*len(line) ,indication
    line,indication=simplify_dots(line,indication)
    c=1
    while c!=0:
        l=len(line)
        line,indication=simplify_left_correct(line,indication)
        line,indication=simplify_right_correct(line,indication)
        c=abs(l-len(line))
    
    return line,indication

#### Function to find if longest indication is already in the string

def simplify_if_longest(line,indication):

    string_to_test='#'*max(indication)
    
    a=1
    while a!=0:
        l=len(line)
        line, indication = simplify_both_correct(line,indication)
        a=abs(l-len(line))

    spans_of_pattern=re.finditer(string_to_test,line)

    if string_to_test in line :
        c=0
        for x in spans_of_pattern:
            
            start_pattern=x.span()[0]
            end_pattern=x.span()[1]

            fin_gauche=start_pattern-1-c*(len(string_to_test)+1)
            debut_droite=end_pattern+1-c*(len(string_to_test)+1)

            line=line[0:fin_gauche]+'.'+ line[debut_droite:]
            c+=1
            index_max=indication.index(max(indication))

            del indication[index_max]
        
    return simplify_both_correct(line,indication)
    
#### Obvious left-right completion when boundaries are # and then #,?

def obvious_left_completion(line,indication):
    line,indication=simplify_if_longest(line,indication)

    if set(line[:indication[0]])=={'#','?'} and line[0]=='#' :
        return line[indication[0]+1:],indication[1:]
    else:
        return line, indication

def obvious_right_completion(line,indication):
    line, indication = simplify_if_longest(line,indication)

    if set(line[-indication[-1]:])=={'#','?'} and line[-1]=='#' :
        return line[:-indication[-1]], indication[:-1]
    
    else:
        return line, indication

def obvious_both_sides_completion(line,indication):
    if line=='':
        return line, indication
    line,indication =simplify_if_longest(line,indication)
    if indication== []:
        return '.'*len(line) , indication
    c=1
    while c!=0:
        l=len(line)
        line,indication = obvious_right_completion(line,indication)
        line,indication = obvious_left_completion(line,indication)
        c=abs(l-len(line))
    return line,indication

    

##### Solver for obvious lines 
def solve_line_if_obvious(line,indication):
    if line=='' or indication==[]:
        return line, indication
    sum_of_indications =np.sum(indication)
    number_dots_necessary=len(indication)-1

    solved_line=''
    
    if sum_of_indications + number_dots_necessary == len(line):
        for x in indication[:-1]:
            solved_line=solved_line+'#'*x+'.'

        solved_line=solved_line+'#'*indication[-1]
        return solved_line, indication
        
    if line.count('#')==sum_of_indications:
        return re.sub('?','.',line),indication

    return line,indication
    

In [5]:
##### All the simplifications above.

def mega_simplifier(line,indication):
    
    new_line, new_indic = simplify_both_correct(line, indication)
    new_line,new_indic = obvious_both_sides_completion(new_line,new_indic)
    new_line,new_indic = solve_line_if_obvious(new_line,new_indic)
    
    temp_input=[(line,indication),(new_line,new_indic)]

    while temp_input[-2]!=temp_input[-1]:
        new_line,new_indic= solve_line_if_obvious(temp_input[-1][0],temp_input[-1][1])
        new_line, new_indic = obvious_both_sides_completion(new_line, new_indic)
        new_line, new_indic = simplify_both_correct(new_line, new_indic)

        temp_input.append((new_line,new_indic))
    return new_line,new_indic
    


        


In [6]:
doc

[['???.###', [1, 1, 3]],
 ['.??..??...?##.', [1, 1, 3]],
 ['?#?#?#?#?#?#?#?', [1, 3, 1, 6]],
 ['????.#...#...', [4, 1, 1]],
 ['????.######..#####.', [1, 6, 5]],
 ['?###????????', [3, 2, 1]]]

In [7]:
### Seeing the whole procedure built

for k, x in enumerate(doc):
    print(x)
    line=doc[k][0]
    indications=doc[k][1]
    new_new_line, new_new_indications = mega_simplifier(line,indications)

    print(new_new_line,new_new_indications,'\n')
    

['???.###', [1, 1, 3]]
 [] 

['.??..??...?##.', [1, 1, 3]]
??..?? [1, 1] 

['?#?#?#?#?#?#?#?', [1, 3, 1, 6]]
?#?#?#?#?#?#?#? [1, 3, 1, 6] 

['????.#...#...', [4, 1, 1]]
 [] 

['????.######..#####.', [1, 6, 5]]
???? [1] 

['?###????????', [3, 2, 1]]
??????? [2, 1] 



In [8]:
doc

[['???.###', [1, 1, 3]],
 ['.??..??...?##.', [1, 1, 3]],
 ['?#?#?#?#?#?#?#?', [1, 3, 1, 6]],
 ['????.#...#...', [4, 1, 1]],
 ['????.######..#####.', [1, 6, 5]],
 ['?###????????', [2, 1]]]

In [9]:
mega_simplifier('???.###', [1, 1, 3])

('', [])

In [10]:
mega_simplifier('.??..??...?##.', [1, 1, 3])

('??..??', [1, 1])